<a href="https://colab.research.google.com/github/mohamad1371/sentiment-analysis-of-twitter-for-crypto-market-using-transformers/blob/main/sentiment_analysis_of_twitter_for_crypto_market%2C_using_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#pip install the libraries, fetching and preprocessing tweets

first  of all we need to clone the repos and install the requirements:

In [1]:
!git clone --depth=1 https://github.com/twintproject/twint.git
%cd twint

!git pull origin master

# install packages requirements
!pip install -r requirements.txt
!pip install aiohttp==3.7.0

# install package
!python setup.py install

#transformers
!pip install transformers

Cloning into 'twint'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 47 (delta 3), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), done.
/content/twint
From https://github.com/twintproject/twint
 * branch            master     -> FETCH_HEAD
Already up to date.
     |████████████████████████████████| 1.1 MB 5.4 MB/s 
     |████████████████████████████████| 263 kB 40.6 MB/s 
     |████████████████████████████████| 385 kB 45.0 MB/s 
     |████████████████████████████████| 192 kB 42.2 MB/s 
     |████████████████████████████████| 271 kB 44.7 MB/s 
     |████████████████████████████████| 160 kB 44.6 MB/s 
     |████████████████████████████████| 291 kB 27.4 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=0ed045b782a8dda8f6e69cd1864754ae74147bc2b6bd6be875326d7cfb1a641a
  Stored in directory: /root/.cache/pip/whee

Then we import the pipeline and define it for sentiment analysis. this pipline was trained on a very large dataset and here we use only the trained model.

In [2]:
from transformers import pipeline

sentiment = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

500 Server Error: Internal Server Error for url: https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json
500 Server Error: Internal Server Error for url: https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json


in the next step we need sth to fetch data from twitter. one way is to use twitter API, which is a little complex and needs things like secrect keys. the other solution is using twint library. it is very easy to use. in the next few lines of code we import and use this library for fetching tweets which sent in last 30 minutes with the keyword bitcoin. we store the tweets in json format. 


In [3]:
import twint
import nest_asyncio
nest_asyncio.apply()  #this 2 lines are for the error "This event loop is already running"

In [4]:
import time
from datetime import datetime
from datetime import datetime, timedelta
last_half_hour = datetime.now() - timedelta(minutes = 30)

last_half_hour = last_half_hour.strftime("%Y-%m-%d %H:%M:%S") #get the exact time/date of last half_hour

# Configure
c = twint.Config()
# c.Username = "github"
c.Search = ''' "bitcoin" '''
c.Since = last_half_hour
# c.Limit = 1010
c.Store_json = True
c.Output  = '/content/json_data'
# Run
twint.run.Search(c)


1473243457951236098 2021-12-21 10:46:26 +0000 <bitcoin_renaiw> オンラインで喧嘩したことない。 オフラインでも喧嘩したことない。
1473243455199584263 2021-12-21 10:46:25 +0000 <BitcoinCity69> @MaddyWithKhan The best service Khan Sb can do to Pakistan. "He should shift his political legacy to the middle class of the country."  PTI is only Imran Khan and no one else seems willing to put effort as he is doing to make Pakistan a better place.
1473243454700408832 2021-12-21 10:46:25 +0000 <AndreasSiemund> Bitcoin
1473243452603260928 2021-12-21 10:46:25 +0000 <fredtheinvestor> @KevinCremer6 Bro, please tell me what is #Bitcoin performance for the past 10 years ?
1473243448660553734 2021-12-21 10:46:24 +0000 <formerhfpm> I'm not even a Bitcoin maxi but Grayscale is a close end fund not an ETF
1473243443673677824 2021-12-21 10:46:23 +0000 <GANNBOX66> #btc  #BTCUSD #Bitcoin #kriptopara #Binance #altcoins  btcusdt 15 kasımdan bu yana devam eden düşen trend yukarı yönde kırıldı kalıcı olabilidği sürece 52400 hedefler  beklentim i

here we read the json file of tweets with pandas:

In [5]:
import pandas as pd

json_data = pd.read_json('/content/json_data/tweets.json', lines=True) #there will be an error if we do not set lines=True

json_data.tail()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
1720,1473235936527978502,1473235936527978496,2021-12-21 10:16:33+00:00,2021-12-21,10:16:33,0,1435846237379497988,toshiwatcher,"toshiwatcher aka ""Busy Child""",,Artwork title: Jimmy the Kid by Artist: Cypher...,en,[],[https://ipfs.io/ipfs/Qmd9sHQVhG2XfHAyMNN8xRyt...,[https://pbs.twimg.com/tweet_video_thumb/FHH8R...,1,0,3,"[nfts, bitcoinart, nftart, bitcoin, btc]",[],https://twitter.com/toshiwatcher/status/147323...,False,,1,https://pbs.twimg.com/tweet_video_thumb/FHH8Rz...,,,,,,,[],,,,
1721,1473235932107128834,1472911479229890560,2021-12-21 10:16:32+00:00,2021-12-21,10:16:32,0,1434453177412341762,franncisco86,Na eleição compre Bitcoin,,@HashtagInsist @gebeleizis616 @holoride @benia...,pt,[],[],[],0,0,0,[],[],https://twitter.com/franncisco86/status/147323...,False,,0,,,,,,,,"[{'screen_name': 'HashtagInsist', 'name': 'GUF...",,,,
1722,1473235922800070658,1473156275261173760,2021-12-21 10:16:30+00:00,2021-12-21,10:16:30,0,1397610058830254081,bitcoin82926764,islandlifeishardto,,@PALNWS Liever ww3 dan een gedwongen vaccinatie 💉,nl,[],[],[],0,0,0,[],[],https://twitter.com/bitcoin82926764/status/147...,False,,0,,,,,,,,"[{'screen_name': 'PALNWS', 'name': 'PAL NWS', ...",,,,
1723,1473235916474839041,1473185433089724416,2021-12-21 10:16:28+00:00,2021-12-21,10:16:28,0,947791025942286336,kaptanmem0,Kaptan Memo,,@Darkwebhaber Doğduklarında harçlık olarak bir...,tr,[],[],[],0,0,0,[],[],https://twitter.com/KaptanMem0/status/14732359...,False,,0,,,,,,,,"[{'screen_name': 'Darkwebhaber', 'name': 'Dark...",,,,
1724,1473235915078258692,1473235915078258688,2021-12-21 10:16:28+00:00,2021-12-21,10:16:28,0,1190775649918689282,cynnefuckinclds,cynne ۵ (celine & soraya love acc),,gucci bitcoin stocks and bacon digga werde ich...,de,[],[],[],2,0,3,[],[],https://twitter.com/cynnefuckinclds/status/147...,False,https://twitter.com/ghostinbia/status/14729282...,0,,,,,,,,[],,,,


in the next five blocks of code we preprocess tweets and hold only the english tweets which are sent from distinct user_ids:

In [6]:
json_data_1 = json_data.drop_duplicates(subset=['user_id']) #only one tweet from each user_id

json_data_1.shape

(1257, 36)

In [7]:
json_data_2 = json_data_1[json_data_1['language']=='en'] #just english tweets

json_data_2.shape

(830, 36)

In [8]:
tweets = json_data_2['tweet'].tolist()

tweets = set(tweets) #to romove duplicate tweets
tweets = list(tweets)
len(tweets)

788

In [9]:
import re
#remove URLS
tweets = [re.sub(r"http\S+", "", i) for i in tweets]

#removing tweets which are less that 3 words
for i in tweets:                                     
    if len(i.split())<3:
        tweets.remove(i)

len(tweets)  

759

In [10]:
tweets = [' '.join(re.sub("@[A-Za-z0-9]+","", i).split()) for i in tweets] #remove @user_names

tweets = [re.sub(r'#([^\s]+)', r'\1', i) for i in tweets] # change #word to word

tweet = [re.sub('[\s]+', ' ', i) for i in tweets] # remove additional whitespaces

tweets

['Bitcoin (BTC) Bull and Bear Runs Still Early in Bubble - The Currency Analytics via BTCnews for iOS',
 "The U.S. government has a massive, secret stockpile of bitcoin — Here's what happens to it",
 'Fiat is slavery. Bitcoin is freedom. Choice wisely',
 '_binance You look great',
 'Bitcoin 1-week AltRank™ is up 13.25% to 94 with price up 2.615% to $48,652.94 $BTC Bitcoin LunarCrush',
 "This is just wrong. Sad that people are retweeting this without doing their own research. That 0.01% are wallets of exchanges where millions of people keep their bitcoin. Yes we know because its open sourced and it's been checked.",
 'Finally $BTC has made a move 🚀 Bitcoin has broken the falling wedge. Now target 55k minimum',
 "Good morning, we'll be LIVE in 60 minutes with our second daily Xmas crypto update. Tune in Bitcoin cryptocurrecy altcoin",
 'Last chances to buy ?! BITCOIN: At this point, with a medium-term outlook, I started buying bitcoins The boring period for Bitcoin buyers is coming to an

#sentiment analysis of tweets with transformers pipeline:


the number of tweets per minute(or number of people tweets per minute) can be a good measure of trading volume. so we calculate it:

In [11]:
import time
from datetime import datetime


end_time_of_tweet = json_data['created_at'].iloc[0]
end_time_of_tweet = end_time_of_tweet.strftime("%Y-%m-%d %H:%M:%S")

start_time_of_tweet = json_data['created_at'].iloc[-1]
start_time_of_tweet = start_time_of_tweet.strftime("%Y-%m-%d %H:%M:%S")

end_time_of_tweet = datetime.strptime(end_time_of_tweet, "%Y-%m-%d %H:%M:%S")
start_time_of_tweet = datetime.strptime(start_time_of_tweet, "%Y-%m-%d %H:%M:%S")

time_distance = abs((end_time_of_tweet - start_time_of_tweet).seconds/60) #time distance in minute

tweets_num = len(json_data)
tweet_rate = '%.2f' % (tweets_num/time_distance)

num_unique_user_ids = len(json_data_1)
ppl_per_hour = '%.2f' % (num_unique_user_ids/time_distance)

print(f'{tweet_rate} tweets per minute') 
#just some examples in different times:
# 55-58 in 10/26/2021 09:00:00 GMT when the price was about 63000 and on peak
# 70 in 2021-10-29 12:37:46 of GMT when 61000
# 95 in 2021-11-03 15:11:27
print(f'{ppl_per_hour} people tweeted per minute') 
#31 in 10/26/2021 09:00:00 GMT
#45 in 2021-10-29 12:37:46 of GMT
#67 in 2021-11-03 15:11:27
print(f'results are taken at the time: {end_time_of_tweet} in GMT')

57.56 tweets per minute
41.95 people tweeted per minute
results are taken at the time: 2021-12-21 10:46:26 in GMT


and finally the sentiment of tweets:

In [12]:
neg_counter = 0
pos_counter = 0

for i in tweets:     #this for-loop will take about 5-10 min to complete

    s = sentiment(i)
    label = s[0]['label']
    score = s[0]['score']
    
    if label == 'POSITIVE':
        pos_counter +=1
    else:
        neg_counter +=1

percent_pos = '%.2f' % ((pos_counter/len(tweets))*100)
percent_neg =  '%.2f' % ((neg_counter/len(tweets))*100)

print(f'- in {len(tweets)} tweets, {percent_pos}% were pos and {percent_neg}% neg.')

#just some examples in different times: 
#about 28% is positive  in 10/27/2021 05:00:00 GMT
#32.74% were pos in 2021-10-29 12:37:46 of GMT
#40.73% were pos in 2021-11-03 15:55:44 

- in 759 tweets, 24.64% were pos and 75.36% neg.
